# 第三次作業

## Basline
將第一次作業的 QA 檢索聊天機器人，重新透過 LangChain 進行實作

In [39]:
from langchain.document_loaders import WebBaseLoader

url = "https://www.google.com/search?q=%E6%88%B4%E8%B3%87%E7%A9%8E%E5%A5%A7%E9%81%8B&sca_esv=84adf72e07b86e49&sca_upv=1&rlz=1C1VDKB_zh-TWTW1042TW1042&biw=1920&bih=919&tbm=nws&sxsrf=ADLYWIIqt57Quw-NSUjoBW48jPc5oifJMQ%3A1720616391025&ei=x4WOZvydAYHt1e8Poauo8Ak&oq=%E5%B8%B6%E5%A7%BF%E7%A9%8E&gs_lp=Egxnd3Mtd2l6LW5ld3MiCeW4tuWnv-epjioCCAEyDRAAGIAEGLEDGIMBGAoyEBAAGIAEGLEDGEMYgwEYigUyDRAAGIAEGLEDGIMBGAoyDRAAGIAEGLEDGIMBGAoyChAAGIAEGLEDGAoyDRAAGIAEGLEDGIMBGAoyDRAAGIAEGLEDGEMYigUyEBAAGIAEGLEDGIMBGIoFGAoyEBAAGIAEGLEDGIMBGIoFGAoyEBAAGIAEGLEDGIMBGIoFGApInxdQAFi1BnAAeACQAQCYATegAZ8DqgEBOLgBA8gBAPgBAZgCCKACsgPCAgUQABiABMICCBAAGIAEGKIEwgIIEAAYgAQYsQPCAgsQABiABBixAxiDAcICBBAAGAPCAg4QABiABBixAxiDARiKBcICBBAAGB7CAgYQABgFGB7CAgcQABiABBgNmAMAkgcBOKAH9RY&sclient=gws-wiz-news"

loader = WebBaseLoader(url)
news_docs = loader.load()

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=30)
texts_chunks = text_splitter.split_documents(news_docs)

In [42]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity


# 初始化 Embedding 模型
embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
from langchain.vectorstores import Chroma
# load it into Chroma
db = Chroma.from_documents(texts_chunks, embedding_func)

In [31]:
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 62915040.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [44]:
# Prompt 模板
template = """
<s>
請你做為一個羽球的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
{context}

{question}
[/INST]
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

result = qa_chain({"query": "誰是戴資穎"})

# LLM 提供的回答
print("回覆：")
print(result["result"])


回覆：

<s>
請你做為一個羽球的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
運動中央社 CNA網球名將謝淑薇退賽與「羽球天后」戴資穎的球鞋爭議，讓2016里約奧運賽場外話題搶走不少運動員的風采，當年靠舉重好手許淑淨奪金帶領，台灣代表團共拿...6 hours ago羽球》戴資穎奧運最後一舞表現受關注 教練賴建誠透露目前狀態自由體育台灣羽球一姐戴資穎今天出席國訓中心的巴黎奧運授旗典禮，雖然她低調婉拒媒體訪問，仍大方和其他選手及與會來賓合照。由於小戴之前有傷在身，...1

戴資穎奧運 - Google SearchGoogle×Please click here if you are not redirected within a few seconds.    AllNewsVideosImages Maps Shopping Books Search tools    RecentRecentPast hourPast 24 hoursPast weekPast

誰是戴資穎
[/INST]
戴資穎是一名來自臺灣的職業羽毛球選手，她是臺灣的女單世界排名第一的球員。她在2016年里約奧運會中獲得銅牌獎牌，並在2021年東京奧運會上奪得銀牌。她被譽為臺灣的「羽球后」，也是臺灣體壇界的重要人物之一。


## Advance
延伸 Basline 的內容，測試看看不同的 chunk 大小，對於生成結果的影響

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=30)
texts_chunks = text_splitter.split_documents(news_docs)

In [47]:
from langchain.vectorstores import Chroma
# load it into Chroma
db = Chroma.from_documents(texts_chunks, embedding_func)

In [48]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

result = qa_chain({"query": "誰是戴資穎"})

# LLM 提供的回答
print("回覆：")
print(result["result"])


回覆：

<s>
請你做為一個羽球的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
運動中央社 CNA網球名將謝淑薇退賽與「羽球天后」戴資穎的球鞋爭議，讓2016里約奧運賽場外話題搶走不少運動員的風采，當年靠舉重好手許淑淨奪金帶領，台灣代表團共拿...6 hours ago羽球》戴資穎奧運最後一舞表現受關注 教練賴建誠透露目前狀態自由體育台灣羽球一姐戴資穎今天出席國訓中心的巴黎奧運授旗典禮，雖然她低調婉拒媒體訪問，仍大方和其他選手及與會來賓合照。由於小戴之前有傷在身，...1

戴資穎確定第3種子進軍巴黎奧運自由體育超級500系列加拿大公開賽落幕後，羽球媒體《Statminton》公布本週最新世界排名，台灣一姊戴資穎持續高居女單第3。戴資穎的積分92221分維持不變，...

誰是戴資穎
[/INST]
戴資穎是一名來自臺灣的職業羽毛球選手，她是臺灣的國家隊成員。她在女子單打項目中獲得了許多成功，包括在國際賽事上贏得多個冠軍頭銜。她被譽為臺灣的「羽球后」，也是臺灣體壇界的重要人物之一。
